In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Francesco Corda\AppData\Local\Temp\ipykernel_1768\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab. Natural Lenguage Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [21]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speead up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [22]:
# Your code
train_data = data
test_data = pd.read_csv("data/kg_test.csv",encoding='latin-1')

In [23]:
display(train_data.shape,test_data.shape)

(1000, 2)

(5964, 1)

In [24]:
train_data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


## Data Preprocessing

In [25]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [26]:
import pandas as pd
import string
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Francesco
[nltk_data]     Corda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Francesco
[nltk_data]     Corda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
# HTML cleaning function
def clean_html(html_content):
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Remove inline JavaScript/CSS
    for script in soup(["script", "style"]):
        script.extract()
    
    # Remove HTML comments
    comments = soup.findAll(text=lambda text: isinstance(text, Comment))
    for comment in comments:
        comment.extract()
    
    # Remove remaining HTML tags
    text = soup.get_text()
    
    return text

In [28]:
# Your code

# Define the preprocessing function
def preprocess_text(text):
    # Remove prefixed 'b' (common when dealing with byte strings)
    text = re.sub(r'\bb\b', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove all special characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Remove all single characters
    text = re.sub(r'\b[a-z]\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'^[a-z]\s+', '', text)
    
    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Stemming
    snowball = SnowballStemmer('english')
    words = [snowball.stem(word) for word in words]
    
    return ' '.join(words)

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [29]:
# Your code

## Now let's work on removing stopwords
Remove the stopwords.

In [30]:
# Your code

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [31]:
# Your code

# Apply preprocessing to the 'email_text' column
train_data['text'] = train_data['text'].apply(preprocess_text)

print(train_data.head())

                                                text  label
0  dear sir strict privat busi propos mike chukwu...      1
1                                                         0
2  noracheryl email dozen memo haiti weekend plea...      0
3  dear sirfmadamc know propos might surpris emer...      1
4                                                fyi      0


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [32]:
# Your code
# Split the dataset into ham and spam
ham_messages = train_data[train_data['label'] == 0]['text']
spam_messages = train_data[train_data['label'] == 1]['text']

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

# Create a function to get top N words from a series of texts
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

# Get the top 10 words in ham messages
top_ham_words = get_top_n_words(ham_messages, 10)
print("Top 10 words in ham messages:", top_ham_words)

# Get the top 10 words in spam messages
top_spam_words = get_top_n_words(spam_messages, 10)
print("Top 10 words in spam messages:", top_spam_words)

Top 10 words in ham messages: [('us', 115), ('pm', 115), ('call', 111), ('would', 106), ('work', 103), ('state', 103), ('presid', 96), ('time', 87), ('obama', 82), ('depart', 77)]
Top 10 words in spam messages: [('money', 937), ('account', 851), ('bank', 842), ('fund', 718), ('us', 549), ('transfer', 535), ('foreign', 519), ('busi', 474), ('contact', 440), ('transact', 417)]


## Extra features

In [37]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

train_data['money_mark'] = train_data['text'].str.contains(money_simbol_list)*1
train_data['suspicious_words'] = train_data['text'].str.contains(suspicious_words)*1
train_data['text_len'] = train_data['text'].apply(lambda x: len(x)) 

test_data['money_mark'] = test_data['text'].str.contains(money_simbol_list)*1
test_data['suspicious_words'] = test_data['text'].str.contains(suspicious_words)*1
test_data['text_len'] = test_data['text'].apply(lambda x: len(x)) 

train_data.head()

,text,label,money_mark,suspicious_words,text_len
0,dear sir strict privat busi propos mike chukwu...,1,1,1,1304
1,,0,1,0,0
2,noracheryl email dozen memo haiti weekend plea...,0,1,0,104
3,dear sirfmadamc know propos might surpris emer...,1,1,1,1212
4,fyi,0,1,0,3


In [38]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   text              1000 non-null   object
 1   label             1000 non-null   int64 
 2   money_mark        1000 non-null   int32 
 3   suspicious_words  1000 non-null   int32 
 4   text_len          1000 non-null   int64 
dtypes: int32(2), int64(2), object(1)
memory usage: 31.4+ KB


## How would work the Bag of Words with Count Vectorizer concept?

In [39]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Step 1: Initialize CountVectorizer
vectorizer = CountVectorizer()

# Step 2: Fit and Transform
X_train_counts = vectorizer.fit_transform(train_data['text'])

# Step 3: Optionally, Convert to DataFrame
dtm_df = pd.DataFrame(X_train_counts.toarray(), columns=vectorizer.get_feature_names_out())

# Concatenate the DTM DataFrame with the original train_data DataFrame
train_data_with_dtm = pd.concat([train_data, dtm_df], axis=1)

# Display the first few rows of the DataFrame with DTM
print(train_data_with_dtm.head())


                                                text  label  money_mark  \
0  dear sir strict privat busi propos mike chukwu...      1           1   
1                                                         0           1   
2  noracheryl email dozen memo haiti weekend plea...      0           1   
3  dear sirfmadamc know propos might surpris emer...      1           1   
4                                                fyi      0           1   

   suspicious_words  text_len  aac  aacloc  aae  aag  aaronovitchon  ...  \
0                 1      1304    0       0    0    0              0  ...   
1                 0         0    0       0    0    0              0  ...   
2                 0       104    0       0    0    0              0  ...   
3                 1      1212    0       0    0    0              0  ...   
4                 0         3    0       0    0    0              0  ...   

   zone  zongo  zongoth  zuhair  zulatoebikozulatonetscapeenet  \
0     0      0        0   

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [40]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Load the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Vectorize the text data
X_train_tfidf = vectorizer.fit_transform(train_data['text'])

# Print the shape of the vectorized dataset
print("Shape of the vectorized dataset:", X_train_tfidf.shape)

Shape of the vectorized dataset: (1000, 17790)


## And the Train a Classifier?

In [41]:
# Your code
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Prepare the Data
X = X_train_tfidf
y = train_data['label']

# Step 2: Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Choose a Classifier (Example: Logistic Regression)
classifier = LogisticRegression()

# Step 4: Train the Classifier
classifier.fit(X_train, y_train)

# Step 5: Evaluate the Classifier
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


Accuracy: 0.935
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.99      0.95       125
           1       0.98      0.84      0.91        75

    accuracy                           0.94       200
   macro avg       0.95      0.92      0.93       200
weighted avg       0.94      0.94      0.93       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code